Untuk menyambungkan Google Drive Anda ke Colab, jalankan kode di bawah ini. Anda akan diminta untuk mengautentikasi dan memberikan izin.

In [54]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [55]:
import pandas as pd

# Path ke file CSV yang baru di Google Drive Anda
file_path_star_data = '/content/drive/MyDrive/star-dataset-to-predict-star-types/6-class.csv'

# Muat file CSV ke dalam DataFrame
df_star = pd.read_csv(file_path_star_data)

In [56]:
df_star.head()

,Temperature (K),Luminosity(L/Lo),Radius(R/Ro),Absolute magnitude(Mv),Star type,Star color,Spectral Class
0,3068,0.002400,0.1700,16.12,0,Red,M
1,3042,0.000500,0.1542,16.60,0,Red,M
2,2600,0.000300,0.1020,18.70,0,Red,M
3,2800,0.000200,0.1600,16.65,0,Red,M
4,1939,0.000138,0.1030,20.06,0,Red,M


In [57]:
df_star.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Temperature (K)         240 non-null    int64  
 1   Luminosity(L/Lo)        240 non-null    float64
 2   Radius(R/Ro)            240 non-null    float64
 3   Absolute magnitude(Mv)  240 non-null    float64
 4   Star type               240 non-null    int64  
 5   Star color              240 non-null    object 
 6   Spectral Class          240 non-null    object 
dtypes: float64(3), int64(2), object(2)
memory usage: 13.3+ KB


In [58]:
df_star.describe()

,Temperature (K),Luminosity(L/Lo),Radius(R/Ro),Absolute magnitude(Mv),Star type
count,240.000000,240.000000,240.000000,240.000000,240.000000
mean,10497.462500,107188.361635,237.157781,4.382396,2.500000
std,9552.425037,179432.244940,517.155763,10.532512,1.711394
min,1939.000000,0.000080,0.008400,-11.920000,0.000000
25%,3344.250000,0.000865,0.102750,-6.232500,1.000000
50%,5776.000000,0.070500,0.762500,8.313000,2.500000
75%,15055.500000,198050.000000,42.750000,13.697500,4.000000
max,40000.000000,849420.000000,1948.500000,20.060000,5.000000


In [59]:
x = df_star.drop('Star type', axis=1)
y = df_star['Star type']

In [60]:
x.head()
y.head()

,Star type
0,0
1,0
2,0
3,0
4,0


In [61]:
x.dtypes

,0
Temperature (K),int64
Luminosity(L/Lo),float64
Radius(R/Ro),float64
Absolute magnitude(Mv),float64
Star color,object
Spectral Class,object


In [62]:
from sklearn.preprocessing import LabelEncoder

le_color = LabelEncoder()
le_spectral = LabelEncoder()

x["Star color"] = le_color.fit_transform(x["Star color"])
x["Spectral Class"] = le_spectral.fit_transform(x["Spectral Class"])

In [63]:
x.head()

,Temperature (K),Luminosity(L/Lo),Radius(R/Ro),Absolute magnitude(Mv),Star color,Spectral Class
0,3068,0.002400,0.1700,16.12,10,5
1,3042,0.000500,0.1542,16.60,10,5
2,2600,0.000300,0.1020,18.70,10,5
3,2800,0.000200,0.1600,16.65,10,5
4,1939,0.000138,0.1030,20.06,10,5


In [64]:
from sklearn.preprocessing import StandardScaler

In [65]:
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

In [66]:
x_scaled = pd.DataFrame(x_scaled, columns=x.columns)
x_scaled.head()

,Temperature (K),Luminosity(L/Lo),Radius(R/Ro),Absolute magnitude(Mv),Star color,Spectral Class
0,-0.779382,-0.598624,-0.459210,1.116745,0.559354,0.595338
1,-0.782110,-0.598624,-0.459241,1.162414,0.559354,0.595338
2,-0.828477,-0.598624,-0.459342,1.362213,0.559354,0.595338
3,-0.807496,-0.598624,-0.459229,1.167171,0.559354,0.595338
4,-0.897819,-0.598624,-0.459340,1.491607,0.559354,0.595338


In [67]:
from sklearn.model_selection import train_test_split


In [68]:
x_train, x_test, y_train, y_test = train_test_split(
    x_scaled,
    y,
    test_size=0.2,
    random_state=42
)

In [69]:
x_train.shape

(192, 6)

In [70]:
y_train.shape

(192,)

In [71]:
import torch
import torch.nn as nn
import torch.optim as optim

In [72]:
x_train_tensor = torch.tensor(x_train.values, dtype=torch.float32)
x_test_tensor = torch.tensor(x_test.values, dtype=torch.float32)

In [73]:
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

In [74]:
class StarClassifier(nn.Module):
  def __init__(self, input_dim, num_classes):
    super().__init__()

    self.fc1 = nn.Linear(input_dim, 16)
    self.fc2 = nn.Linear(16, 16)
    self.fc3 = nn.Linear(16, num_classes)

  def forward(self, x):
    x = torch.relu(self.fc1(x))
    x = torch.relu(self.fc2(x))
    x = self.fc3(x)
    return (x)

In [75]:
input_dim = x_train.shape[1]
num_classes = len(torch.unique(y_train_tensor))
model = StarClassifier(input_dim, num_classes)

In [76]:
criterion = nn.CrossEntropyLoss()

In [77]:
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [78]:
epochs = 500

In [79]:
for epoch in range(epochs):

    optimizer.zero_grad()

    outputs = model(x_train_tensor)

    loss = criterion(outputs, y_train_tensor)

    loss.backward()

    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")


Epoch 10/500, Loss: 1.8047
Epoch 20/500, Loss: 1.7743
Epoch 30/500, Loss: 1.7454
Epoch 40/500, Loss: 1.7154
Epoch 50/500, Loss: 1.6819
Epoch 60/500, Loss: 1.6430
Epoch 70/500, Loss: 1.5964
Epoch 80/500, Loss: 1.5400
Epoch 90/500, Loss: 1.4689
Epoch 100/500, Loss: 1.3834
Epoch 110/500, Loss: 1.2868
Epoch 120/500, Loss: 1.1854
Epoch 130/500, Loss: 1.0857
Epoch 140/500, Loss: 0.9926
Epoch 150/500, Loss: 0.9097
Epoch 160/500, Loss: 0.8374
Epoch 170/500, Loss: 0.7725
Epoch 180/500, Loss: 0.7137
Epoch 190/500, Loss: 0.6601
Epoch 200/500, Loss: 0.6108
Epoch 210/500, Loss: 0.5656
Epoch 220/500, Loss: 0.5248
Epoch 230/500, Loss: 0.4883
Epoch 240/500, Loss: 0.4555
Epoch 250/500, Loss: 0.4261
Epoch 260/500, Loss: 0.3992
Epoch 270/500, Loss: 0.3744
Epoch 280/500, Loss: 0.3513
Epoch 290/500, Loss: 0.3300
Epoch 300/500, Loss: 0.3097
Epoch 310/500, Loss: 0.2899
Epoch 320/500, Loss: 0.2712
Epoch 330/500, Loss: 0.2539
Epoch 340/500, Loss: 0.2378
Epoch 350/500, Loss: 0.2226
Epoch 360/500, Loss: 0.2082
E

In [80]:
model.eval()

StarClassifier(
  (fc1): Linear(in_features=6, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=6, bias=True)
)

In [81]:
with torch.no_grad():
  outputs = model(x_test_tensor)

In [82]:
_, predicted = torch.max(outputs, 1)

In [83]:
correct = (predicted == y_test_tensor).sum().item()

In [84]:
accuracy = correct / y_test_tensor.size(0)
print(f"Test Accuracy: {accuracy:.2%}")

Test Accuracy: 97.92%


In [85]:
from sklearn.metrics import confusion_matrix, classification_report

In [86]:
y_true = y_test_tensor.numpy()
y_pred = predicted.numpy()

In [87]:
cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(cm)

Confusion Matrix:
[[ 8  0  0  0  0  0]
 [ 0  7  0  0  0  0]
 [ 0  0  6  0  0  0]
 [ 0  1  0  7  0  0]
 [ 0  0  0  0  8  0]
 [ 0  0  0  0  0 11]]


In [88]:
print("\nClassification Report:\n", classification_report(y_true, y_pred))


Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       0.88      1.00      0.93         7
           2       1.00      1.00      1.00         6
           3       1.00      0.88      0.93         8
           4       1.00      1.00      1.00         8
           5       1.00      1.00      1.00        11

    accuracy                           0.98        48
   macro avg       0.98      0.98      0.98        48
weighted avg       0.98      0.98      0.98        48



In [89]:
torch.save(model.state_dict(), '/content/drive/MyDrive/star-dataset-to-predict-star-types/star_classifier_model.pth')

In [90]:
print(model.state_dict())

OrderedDict({'fc1.weight': tensor([[ 5.3463e-03,  3.8082e-01, -9.1969e-01, -5.5785e-01, -3.9146e-01,
          5.2596e-01],
        [-3.4808e-01, -7.2722e-01, -5.0742e-01,  1.0633e-01,  1.8627e-01,
         -6.8018e-01],
        [-1.2374e-01,  4.4098e-02,  2.1061e-01, -2.3400e-01,  7.7046e-01,
         -6.8029e-01],
        [ 4.2834e-01,  5.3303e-01, -6.7097e-01,  9.9523e-01,  1.2350e-01,
         -6.9512e-01],
        [-5.9697e-01, -6.9011e-01, -1.1204e-01,  4.4318e-01,  2.2884e-01,
          3.7332e-01],
        [ 5.4229e-01, -4.8216e-02,  4.5693e-01, -8.5574e-01,  3.2565e-01,
         -5.1065e-01],
        [ 5.7350e-01, -4.9807e-01, -1.2407e-01,  7.7976e-01, -5.7306e-01,
         -2.4207e-01],
        [-4.6489e-01,  4.3260e-01, -3.2019e-01, -4.1672e-01,  2.0952e-01,
          3.2333e-01],
        [-1.6452e-01, -1.7458e-03,  5.8350e-01, -4.6684e-01,  1.1252e-01,
          1.4739e-01],
        [-5.6281e-01,  2.5653e-01, -5.9604e-01,  2.5512e-01, -2.3894e-04,
          6.5831e-01],
   

In [96]:
import joblib

joblib.dump(scaler, "scaler.pkl")
joblib.dump(le_color, "color_encoder.pkl")
joblib.dump(le_spectral, "spectral_encoder.pkl")

['spectral_encoder.pkl']